In [1]:
import gensim.downloader as api
#dataset = api.load("text8")
import math
import numpy as np
import h5py
import sys
sys.path.append("..")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"    
import tensorflow as tf
from Vocabulary import *
import time
tf.keras.backend.clear_session()
import csv
import random

In [2]:
class CSV_writer:
    def __init__(self,name,mode):
        self.csvfile = open('{name}.csv'.format(name=name), mode, newline='') 
        fieldnames = ['Optimizer', 'learning_rate','epoch','loss']
        self.writer = csv.DictWriter(self.csvfile, fieldnames=fieldnames)
        if(mode == "w"):
            self.writer.writeheader()
    
    def write(self,opt,learning_rate,epoch,loss):
        lr = "{x:.6e}".format(x=learning_rate)
        loss = int(loss)
        self.writer.writerow({'Optimizer': opt, 'learning_rate': lr,'epoch':epoch,'loss':loss})
        
    
    

In [3]:

class ModelTrainer:
    def __init__(self,vocab,amount_split,vector_size = 300):
        self.vector_size = vector_size
        self.tf_bias = None
        self.tf_context_bias = None
        self.weights = None
        self.tf_context_weights = None
        self.tf_co_occurences = None
        self.zeilen = 0
        self.spalten = 0
        self.amount_split = amount_split
        self.block_length = math.ceil(vocab.get_size()/amount_split)

    
    def prepare(self,filename):
        self.f = h5py.File("S:\\{filename}.hdf5".format(filename=filename), "w")#plus experiment name
        #initalize all the HDF files
        self.HDF_weights = self.f.create_dataset("weights", (vocab_size, self.vector_size))
        self.HDF_context_weights = self.f.create_dataset("context-weights",(self.vector_size,vocab_size))
        self.HDF_bias = self.f.create_dataset("bias", (vocab_size,1))
        self.HDF_context_bias = self.f.create_dataset("context_bias", (1,vocab_size))

        self.hf = h5py.File("S:\\text8-filtered100-coocurences.hdf5", "r")
        self.HDF_coocurrence = self.hf.get('matrix')
        self.init_matrices()
        
    
    def init_matrices(self,chunk_size=10000):
        random.seed(1)
        np.random.seed(1)
        tf.random.set_seed(1)
        self.init_hdf_matrix(self.HDF_weights,-0.5,0.5,chunk_size)
        self.init_hdf_matrix(self.HDF_context_weights,-0.5,0.5,chunk_size)
        self.init_hdf_matrix(self.HDF_bias,-0.5,0.5,chunk_size)
        self.init_hdf_matrix(self.HDF_context_bias,-0.5,0.5,chunk_size)
    
    def init_hdf_matrix(self,hdf_data,min_value,max_value,block_length):
        if len(hdf_data) > len(hdf_data[0]):
            iterations = int(math.ceil(len(hdf_data) / block_length))
            for i in range(iterations):
                current_size = min(block_length,len(hdf_data)-block_length*i)
                hdf_data[i*block_length:(i+1)*block_length , :] = np.random.rand(current_size,len(hdf_data[0]))/self.vector_size
        else:
            iterations = int(math.ceil(len(hdf_data[0]) / block_length))
            for i in range(iterations):
                current_size = min(block_length,len(hdf_data[0])-block_length*i)
                hdf_data[:,i*block_length:(i+1)*block_length] = np.random.rand(len(hdf_data),current_size)/self.vector_size
            
    
    def load_blocks(self,zeilen,spalten):
        co_ocurences = self.HDF_coocurrence[zeilen*self.block_length:(zeilen+1)*self.block_length,spalten*self.block_length:(spalten+1)*self.block_length]
    
        self.tf_co_occurences = tf.convert_to_tensor(co_ocurences,dtype=tf.dtypes.float32)
        co_occurence = None
        #Use normal matrix, if epsilon Shift, than add one to co-ocurence table to fix scaling and log
        self.tf_bias = tf.Variable(initial_value=self.HDF_bias[zeilen*self.block_length:(zeilen+1)*self.block_length,:],dtype=tf.dtypes.float32)
        self.tf_context_bias = tf.Variable(initial_value=self.HDF_context_bias[:,spalten*self.block_length:(spalten+1)*self.block_length],dtype=tf.dtypes.float32)
        self.tf_weights =  tf.Variable(initial_value=self.HDF_weights[zeilen*self.block_length:(zeilen+1)*self.block_length,:],dtype=tf.dtypes.float32)
        self.tf_context_weights = tf.Variable(initial_value=self.HDF_context_weights[:,spalten*self.block_length:(spalten+1)*self.block_length],dtype=tf.dtypes.float32)
    
    def save_blocks(self,zeilen,spalten):
        self.HDF_bias[zeilen*self.block_length:(zeilen+1)*self.block_length,:] = self.tf_bias.numpy()
        self.HDF_context_bias[0,spalten*self.block_length:(spalten+1)*self.block_length] = self.tf_context_bias.numpy()
        self.HDF_weights[zeilen*self.block_length:(zeilen+1)*self.block_length,:] = self.tf_weights.numpy()
        self.HDF_context_weights[:,spalten*self.block_length:(spalten+1)*self.block_length] = self.tf_context_weights.numpy()
    
    def _close_files(self):
        self.f.close()
        self.hf.close()
    
    def train_splitted(self,epochs,optimizer):
        self.epsilon = tf.constant(1e-5) * tf.ones((self.block_length,self.block_length),dtype=tf.dtypes.float32)

        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        0.01,
        decay_steps= self.amount_split*self.amount_split * 10,
        decay_rate=0.5,
        staircase=True)
        
        #print(optimizer._learning_rate)
        print(optimizer.learning_rate)
        
        for epoch in range(0,epochs):
            cur_loss = 0
            for zeilen in range(0,self.amount_split):#CHANGE THIS BACK !!!!!!!!!!!!!!!!!!!
                for spalten in range(0,self.amount_split):
                    tic = time.perf_counter()
                    #print("zeilen"+str(zeilen)+"spalten"+str(spalten))
                    self.zeilen = zeilen
                    self.spalten = spalten
                
                    self.load_blocks(zeilen,spalten)

                    train = optimizer.minimize(self.loss, var_list=[self.tf_bias,self.tf_context_bias,
                                                              self.tf_weights,self.tf_context_weights])
                    cur_loss += self.loss()
        
                    self.save_blocks(zeilen,spalten)
            print("epoch" + str(epoch)+"loss:"+str(cur_loss.numpy()))
            csv_writer.write(optimizer.get_config()['name'],optimizer.learning_rate.numpy(),epoch+1,cur_loss.numpy())
        self._close_files()
        return None

    def loss(self):
        ones_symetrical = tf.ones((self.block_length,self.block_length), dtype=tf.dtypes.float32, name=None)
    
        co_occurences = None
        #Append zero matrices if necessary
        if(self.zeilen == self.amount_split - 1):
            difference = self.block_length - self.tf_bias.shape[0]
            add_to_bias   = tf.zeros((difference,1),dtype=tf.dtypes.float32)
            add_to_co     = tf.zeros((difference,self.tf_context_bias.shape[1]),dtype=tf.dtypes.float32)
            co_occurences = tf.concat([self.tf_co_occurences,add_to_co],axis=0)
            add2_weights  = tf.zeros((difference,self.vector_size),dtype=tf.dtypes.float32)
            weights       = tf.concat([self.tf_weights,add2_weights],axis = 0)
            bias_matrix   = tf.concat([self.tf_bias,add_to_bias],axis = 0) * ones_symetrical
        else:
            bias_matrix   = self.tf_bias * ones_symetrical
            weights       = self.tf_weights
            co_occurences = self.tf_co_occurences
    
        if(self.spalten == self.amount_split - 1):
            difference = self.block_length - self.tf_context_bias.shape[1]
            add_to_con_bias = tf.zeros((1,difference),dtype=tf.dtypes.float32)
            add_to_co       = tf.zeros((self.block_length,difference),dtype=tf.dtypes.float32)
            if co_occurences == None:
                co_occurences   = tf.concat([self.tf_co_occurences,add_to_co],axis=1)
            else:
                co_occurences   = tf.concat([co_occurences,add_to_co],axis=1)
                add2_co_weights = tf.zeros((self.vector_size,difference),dtype=tf.dtypes.float32)
                context_weights = tf.concat([self.tf_context_weights,add2_co_weights],axis = 1)
                context_bias_matrix = tf.concat([self.tf_context_bias,add_to_con_bias],axis=1) * ones_symetrical
        else:
            if co_occurences == None:
                co_occurences   = self.tf_co_occurences
            context_weights     = self.tf_context_weights
            context_bias_matrix = self.tf_context_bias * ones_symetrical
          
                                                          
        bias_terms = context_bias_matrix + bias_matrix
    
        weight_matrix = tf.matmul(weights,context_weights)
        log_X = tf.math.log(co_occurences + self.epsilon)
        inner_sum = bias_terms + weight_matrix - log_X
        squared_sum = tf.math.square(inner_sum)
        weighted_sum = self.cut_function2(co_occurences) * squared_sum
        reduced = tf.math.reduce_sum(weighted_sum)
        return reduced
    
    alpha = tf.constant(0.75,dtype=tf.dtypes.float32)
    XMAX = tf.constant(100.0,dtype=tf.dtypes.float32)  
    def cut_function2(self,value):
        clipped = tf.clip_by_value(value, clip_value_min = 0.0, clip_value_max=100.0)
        return tf.pow(clipped / self.XMAX, self.alpha)

In [4]:
vocab = Vocabulary()
vocab.load('../vocabs/text8_filtered100')
vocab_size = len(vocab.word2Id)
csv_writer = CSV_writer('lrFinderNeu','a+')

learning_rate_step = 0.0001
lr =  0.0352#init
#tf.keras.optimizers.Adam(),tf.keras.optimizers.Adamax(),tf.keras.optimizers.RMSprop(),tf.keras.optimizers.Adadelta(),tf.keras.optimizers.Adagrad(),tf.keras.optimizers.Ftrl(),tf.keras.optimizers.Nadam(),tf.keras.optimizers.SGD()]
    

while lr < 0.3:
    tf.keras.backend.clear_session()
    opt = tf.keras.optimizers.Adam()
    print(opt)
    trainer = ModelTrainer(vocab,1)
    opt.learning_rate = lr
    trainer.prepare('lrFinderNeu')
    print(opt.get_config())
    trainer.train_splitted(3,opt)
    csv_writer.csvfile.flush()
    lr = lr + learning_rate_step

csv_writer.csvfile.close()


{'name': 'Adam', 'learning_rate': 0.0352, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.0352>
epoch0loss:5159665.5
epoch1loss:3439379.2
epoch2loss:2787252.8
{'name': 'Adam', 'learning_rate': 0.035300000000000005, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.0353>
epoch0loss:5153031.5
epoch1loss:3429963.8
epoch2loss:2782842.5
{'name': 'Adam', 'learning_rate': 0.03540000000000001, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.0354>
epoch0loss:5146392.5
epoch1loss:3420604.8
epoch2loss:2778201.0
{'name': 'Adam', 'learning_rate': 0.03550000000000001, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.0355>

epoch0loss:5005527.0
epoch1loss:3238111.5
epoch2loss:2652492.0
{'name': 'Adam', 'learning_rate': 0.03760000000000007, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.0376>
epoch0loss:4998758.0
epoch1loss:3230145.8
epoch2loss:2645459.5
{'name': 'Adam', 'learning_rate': 0.037700000000000074, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.0377>
epoch0loss:4991984.0
epoch1loss:3222250.8
epoch2loss:2639022.8
{'name': 'Adam', 'learning_rate': 0.03780000000000008, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.0378>
epoch0loss:4985203.5
epoch1loss:3214427.2
epoch2loss:2633051.5
{'name': 'Adam', 'learning_rate': 0.03790000000000008, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad':

<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.0399>
epoch0loss:4841812.0
epoch1loss:3067866.5
epoch2loss:2467913.2
{'name': 'Adam', 'learning_rate': 0.04000000000000014, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.04>
epoch0loss:4834941.5
epoch1loss:3061790.8
epoch2loss:2459006.2
{'name': 'Adam', 'learning_rate': 0.04010000000000014, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.0401>
epoch0loss:4828070.0
epoch1loss:3055802.0
epoch2loss:2453329.0
{'name': 'Adam', 'learning_rate': 0.040200000000000145, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.0402>
epoch0loss:4821193.0
epoch1loss:3049901.5
epoch2loss:2445792.5
{'name': 'Adam', 'learning_rate': 0.04030000000000015, 'decay'

epoch0loss:4676224.0
epoch1loss:2947638.8
epoch2loss:2332318.2
{'name': 'Adam', 'learning_rate': 0.04240000000000021, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.0424>
epoch0loss:4669301.0
epoch1loss:2943862.0
epoch2loss:2329062.2
{'name': 'Adam', 'learning_rate': 0.04250000000000021, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.0425>
epoch0loss:4662376.5
epoch1loss:2940189.5
epoch2loss:2324870.0
{'name': 'Adam', 'learning_rate': 0.042600000000000214, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.0426>
epoch0loss:4655450.5
epoch1loss:2936623.0
epoch2loss:2321734.5
{'name': 'Adam', 'learning_rate': 0.04270000000000022, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad':

<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.0447>
epoch0loss:4509907.5
epoch1loss:2887503.5
epoch2loss:2293914.0
{'name': 'Adam', 'learning_rate': 0.04480000000000028, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.0448>
epoch0loss:4502980.5
epoch1loss:2886457.5
epoch2loss:2292708.0
{'name': 'Adam', 'learning_rate': 0.04490000000000028, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.0449>
epoch0loss:4496052.0
epoch1loss:2885534.0
epoch2loss:2292737.8
{'name': 'Adam', 'learning_rate': 0.04500000000000028, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.045>
epoch0loss:4489125.0
epoch1loss:2884735.5
epoch2loss:2295256.5
{'name': 'Adam', 'learning_rate': 0.045100000000000286, 'decay

...

<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.2316>
epoch0loss:180923150.0
epoch1loss:3558947.5
epoch2loss:104585050.0
{'name': 'Adam', 'learning_rate': 0.2316999999999908, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.2317>
epoch0loss:181268370.0
epoch1loss:3557471.0
epoch2loss:104743400.0
{'name': 'Adam', 'learning_rate': 0.2317999999999908, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.2318>
epoch0loss:181614060.0
epoch1loss:3555996.5
epoch2loss:104904616.0
{'name': 'Adam', 'learning_rate': 0.23189999999999078, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.2319>
epoch0loss:181960320.0
epoch1loss:3554520.0
epoch2loss:105060030.0
{'name': 'Adam', 'learning_rate': 0.23199999999

<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.234>
epoch0loss:189342270.0
epoch1loss:3523781.8
epoch2loss:107995170.0
{'name': 'Adam', 'learning_rate': 0.23409999999999054, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.2341>
epoch0loss:189699150.0
epoch1loss:3522330.8
epoch2loss:108121910.0
{'name': 'Adam', 'learning_rate': 0.23419999999999053, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.2342>
epoch0loss:190056500.0
epoch1loss:3520880.8
epoch2loss:108234880.0
{'name': 'Adam', 'learning_rate': 0.23429999999999052, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.2343>
epoch0loss:190414430.0
epoch1loss:3519430.8
epoch2loss:108343980.0
{'name': 'Adam', 'learning_rate': 0.2343999999

<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.2364>
epoch0loss:198043820.0
epoch1loss:3489234.2
epoch2loss:111000470.0
{'name': 'Adam', 'learning_rate': 0.23649999999999027, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.2365>
epoch0loss:198412660.0
epoch1loss:3487807.8
epoch2loss:111106150.0
{'name': 'Adam', 'learning_rate': 0.23659999999999026, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.2366>
epoch0loss:198781970.0
epoch1loss:3486384.0
epoch2loss:111215640.0
{'name': 'Adam', 'learning_rate': 0.23669999999999025, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.2367>
epoch0loss:199151700.0
epoch1loss:3484961.8
epoch2loss:111331050.0
{'name': 'Adam', 'learning_rate': 0.236799999

<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.2388>


KeyboardInterrupt: 